## Byte-Pair Encoding (BPE) Tokenizer

### 2.1 The Unicode Standard

**a. What Unicode character does chr(0) return?**

'\x00'

**b. How does this character’s string representation (__repr__()) differ from its printed representation?**

It printed as null.

**c. What happens when this character occurs in text? It may be helpful to play around with the
following in your Python interpreter and see if it matches your expectations:**

```
>>> chr(0)
'\x00'
>>> chr(0).__repr__()
"'\\x00'"
>>> chr(0)
'\x00'
>>> print(chr(0))

>>> "this is a test" + chr(0) + "string"
'this is a test\x00string'
>>> print("this is a test" + chr(0) + "string")
this is a teststring
```

### 2.2 Unicode Encodings

**a. What are some reasons to prefer training our tokenizer on UTF-8 encoded bytes, rather than
UTF-16 or UTF-32? It may be helpful to compare the output of these encodings for various
input strings.**

| Character | Description | UTF-8 (bytes)         | UTF-16 (bytes)        | UTF-32 (bytes)        |
| --------- | ----------- | --------------------- | --------------------- | --------------------- |
| `A`       | ASCII       | `0x41`                | `0x00 0x41`           | `0x00 0x00 0x00 0x41` |
| `ñ`       | Latin-1     | `0xC3 0xB1`           | `0x00 0xF1`           | `0x00 0x00 0x00 0xF1` |
| `中`       | CJK         | `0xE4 0xB8 0xAD`      | `0x4E 0x2D`           | `0x00 0x00 0x4E 0x2D` |
| `😀`      | Emoji       | `0xF0 0x9F 0x98 0x80` | `0xD8 0x3D 0xDE 0x00` | `0x00 0x01 0xF6 0x00` |
 

1. UTF-8 is more compact for common characters (ASCII)
2. Each byte is in [0, 255] → fixed vocabulary size.

**b. Consider the following (incorrect) function, which is intended to decode a UTF-8 byte string into
a Unicode string. Why is this function incorrect? Provide an example of an input byte string
that yields incorrect results.**

```
def decode_utf8_bytes_to_str_wrong(bytestring: bytes):
return ""
.join([bytes([b]).decode("utf-8") for b in bytestring])
>>> decode_utf8_bytes_to_str_wrong("hello".encode("utf-8"))
'hello'
```

```
>>> decode_utf8_bytes_to_str_wrong("中".encode("utf-8"))
Traceback (most recent call last):
  File "<python-input-31>", line 1, in <module>
    decode_utf8_bytes_to_str_wrong("中".encode("utf-8"))
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^
  File "<python-input-28>", line 2, in decode_utf8_bytes_to_str_wrong
    return "".join([bytes([b]).decode("utf-8") for b in bytestring])
                    ~~~~~~~~~~~~~~~~~^^^^^^^^^
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe4 in position 0: unexpected end of data
```

The special characters need to be decoded through a sequence of bytes rather than one single byte.


**c. Give a two byte sequence that does not decode to any Unicode character(s).**

```
>>> b'\xe3\xe4'.decode("utf-8")
Traceback (most recent call last):
  File "<python-input-32>", line 1, in <module>
    b'\xe3\xe4'.decode("utf-8")
    ~~~~~~~~~~~~~~~~~~^^^^^^^^^
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe3 in position 0: invalid continuation byte
```

Missing the required 2 continuation bytes for 0xE3, and 0xE4 is not a valid follow-up byte.


### 2.5 Experimenting with BPE Tokenizer Training

In [4]:
# Load TinyStories train data
with open("./data/TinyStoriesV2-GPT4-train.txt") as file:
    words = file.read().split(',')
    
display(words[:5])

['\nOnce upon a time there was a little boy named Ben. Ben loved to explore the world around him. He saw many amazing things',
 ' like beautiful vases that were on display in a store. One day',
 ' Ben was walking through the store when he came across a very special vase. When Ben saw it he was amazed!  \nHe said',
 ' “Wow',
 ' that is a really amazing vase! Can I buy it?” \nThe shopkeeper smiled and said']

In [12]:
# Use example util to find the chunk boundary which is required when doing the pre-tokenization in parallel
import os
from typing import BinaryIO

def find_chunk_boundaries(
    file: BinaryIO, 
    desired_num_chunks: int, 
    split_special_token: bytes
) -> list[int]:
    """
    Chunk the file into parts that can be counted independently.
    May return fewer chunks if the boundaries end up overlapping.
    """
    assert isinstance(split_special_token, bytes), (
        "Must represent special token as a bytestring"
    )

    # Get total file size in bytes
    file.seek(0, os.SEEK_END)
    file_size = file.tell()
    file.seek(0)

    chunk_size = file_size // desired_num_chunks

    # Initial guesses for chunk boundary locations, uniformly spaced
    # Chunks start on previous index, don't include last index
    chunk_boundaries = [i * chunk_size for i in range(desired_num_chunks + 1)]
    chunk_boundaries[-1] = file_size

    mini_chunk_size = 4096  # Read ahead by 4k bytes at a time

    for bi in range(1, len(chunk_boundaries) - 1):
        initial_position = chunk_boundaries[bi]
        file.seek(initial_position)  # Start at boundary guess
        while True:
            mini_chunk = file.read(mini_chunk_size)  # Read a mini chunk

            # If EOF, this boundary should be at the end of the file
            if mini_chunk == b"":
                chunk_boundaries[bi] = file_size
                break

            # Find the special token in the mini chunk
            found_at = mini_chunk.find(split_special_token)
            if found_at != -1:
                chunk_boundaries[bi] = initial_position + found_at
                break
            initial_position += mini_chunk_size

    # Make sure all boundaries are unique, but might be fewer than desired_num_chunks
    return sorted(set(chunk_boundaries))